In [ ]:
import random
import pandas as pd
from sage.graphs.graph_generators import graphs 


grafi = []  


# za generirajnje grafov najprej potrebujemo definirati cikle in drevesa:
def cikel(n):
    G = Graph()
    G.add_vertices(range(n))
    e = [(i, (i + 1) % n) for i in range(n)]
    G.add_edges(e)
    return G

def drevo(i):
    return [Graph(tree) for tree in graphs.trees(i)]
 
for i in range(2, 12):  
    grafi.append(cikel(i))
    grafi.extend(drevo(i))


# 2. možnost generiranja grafov - več časa in prostora potrebuje za generiranje grafov:    
# Zanka za generiranje grafov za število vozlišč od 1 do 8 (velja, da je zgornja meja (n+1), kjer je n največe število vozlišč, ki jih lahko ima graf) 
# lahko bi izbrali tudi večji n, vendar se mi v CoClacu program ustavi, saj je mu zmanjka prostora za shranjevanje

# for i in range(1, 9):
#    for G in graphs.nauty_geng(f'{i} -c'):  # 'c' pomeni povezane grafe
#        # Dodaj graf G v seznam grafov
#        grafi.append(G)


###########################################################

def SDCTD_stevilo2(g):
    h = g.complement()

    p = MixedIntegerLinearProgram(maximization = False)
    x = p.new_variable(binary = True)
    p.set_objective(sum([x[v] for v in g]))

    for v in g.vertices():
        neighbors = g.neighbors(v)  
        p.add_constraint(x[v] + sum(x[w] for w in neighbors) >= 1)

    zaustavi = False

    for v in g.vertices():
        neighbors_complement = h.neighbors(v)  
        if neighbors_complement:  
            p.add_constraint(sum(x[w] for w in neighbors_complement) >= 1)

        else: 
            zaustavi = True
            break 

    if not zaustavi == True:
        p.solve()
        x = p.get_values(x)
        return(sum(1 for i in x.values() if i == 1))
    else:
        return None
    
############################################################ 

rezultati = []

for n in range(1, 1000):
    g1 = random.choice(grafi)
    g2 = random.choice(grafi)
    # Kartezični produkt med g1 in g2
    g3 = g1.cartesian_product(g2)

    # Izračun SDCTD števila za vsak graf
    SDCTD_stevilo_graf1 = SDCTD_stevilo2(g1)
    SDCTD_stevilo_graf2 = SDCTD_stevilo2(g2)
    SDCTD_stevilo_graf3 = SDCTD_stevilo2(g3)

    # Dodajanje rezultatov v seznam
    rezultati.append({
        'Izbira grafov': f'Izbira {n}',
        'SDCTD_graf1': SDCTD_stevilo_graf1,
        'SDCTD_graf2': SDCTD_stevilo_graf2,
        'SDCTD_kartezicni': SDCTD_stevilo_graf3
    })

df = pd.DataFrame(rezultati)

# Funkcija za preverjanje posamezne neenačbe
def preveri_neenacbo(df, faktor, oznaka):
    def neenacba(row):
        if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:
            return True
        # Preveri, če velja pogoj za trenutni faktor
        if row['SDCTD_graf1'] * row['SDCTD_graf2'] <= faktor * row['SDCTD_kartezicni']:
            return True
        return False

    # Dodajanje stolpca za trenutno neenačbo
    df[oznaka] = df.apply(neenacba, axis=1)

    # Prešteje število False vrednosti
    false_count = (df[oznaka] == False).sum()

    # Izračuna delež False vrednosti
    false_ratio = false_count / len(df)

    print(f"Za {oznaka}:")
    print(f"Število False v stolpcu {oznaka}: {false_count}")
    print(f"Delež False v stolpcu {oznaka}: {false_ratio:.2%}")
    print()

# Seznam faktorjev za preverjanje
faktorji = [
    (1, 'neenacba1'),
    (2, 'neenacba2'),
    (3, 'neenacba3')
]

for faktor, oznaka in faktorji:
    preveri_neenacbo(df, faktor, oznaka)



Za neenacba1:
Število False v stolpcu neenacba1: 9
Delež False v stolpcu neenacba1: 0.90%

Za neenacba2:
Število False v stolpcu neenacba2: 0
Delež False v stolpcu neenacba2: 0.00%

Za neenacba3:
Število False v stolpcu neenacba3: 0
Delež False v stolpcu neenacba3: 0.00%



In [ ]:
# preverimo, kaj se zgodi, če fiksiranmo g1, g2 pa je poljuben - Vizingova domneva:
rezultati = []

g1 = random.choice(grafi)   # fiksiramo graf1

for m in range(1, 500):
    g2 = random.choice(grafi)
    g3 = g1.cartesian_product(g2)
    
    SDCTD_stevilo_graf1 = SDCTD_stevilo2(g1)
    SDCTD_stevilo_graf2 = SDCTD_stevilo2(g2)
    SDCTD_stevilo_graf3 = SDCTD_stevilo2(g3)
    
    rezultati.append({
        'SDCTD_graf1': SDCTD_stevilo_graf1,
        'SDCTD_graf2': SDCTD_stevilo_graf2,
        'SDCTD_kartezicni': SDCTD_stevilo_graf3
    })


    df = pd.DataFrame(rezultati)


    def vizingova_domneva(row):
        if row['SDCTD_graf1'] is None or row['SDCTD_graf2'] is None:
            return True
        # Preveri, če velja pogoj za produkt in kartezično vrednost
        if row['SDCTD_graf1'] * row['SDCTD_graf2'] <= row['SDCTD_kartezicni']:
            return True
        return False

df['Vizingova domneva'] = df.apply(vizingova_domneva, axis=1)

# pogleda, koliko False se pojavi, torej v koliko primerih ne velja neenačba:
false_vizingova_domneva = (df['Vizingova domneva'] == False).sum()

print(f"Število False v stolpcu Vizingova domneva: {false_vizingova_domneva}")


Število False v stolpcu Vizingova domneva: 0
